ESPN base 建模資料前處理

In [1]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import requests as rs
from bs4 import BeautifulSoup

# 讀取ESPN FA 與表現合併

## 讀取2006~2019年 簽約資料

In [2]:
ESPN_FA_path = r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_FreeAgent_2006to2019_20210417_12.00.34.csv"
ESPN_FA_df = read_data_dir_fPathExt(ESPN_FA_path)

  >> 讀取:: G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_FreeAgent_2006to2019_20210417_12.00.34.csv


In [3]:
ESPN_FA_df.rename(columns={"PLAYER_id":"playerID", "YRS":"contract_Yrs"}, inplace=True)

In [4]:
# ESPN_FA_df["contract_Yrs"].value_counts()
# ESPN_FA_df[ESPN_FA_df["contract_Yrs"].isin([13])]

In [40]:
# ESPN_FA_df[ESPN_FA_df["PLAYER"].str.contains("Moo")]
# ESPN_FA_df[ESPN_FA_df["playerID"].isin(["28525"])]

,_idx,PLAYER,POS,AGE,STATUS,ORG TEAM,NEW TEAM,contract_Yrs,RK,DOLLARS,YEAR,link,is_PLAYER_link_check,playerID,ORG_TEAM_ID,NEW_TEAM_ID
0,0,Matt Albers,RP,38.0,FA,--,--,NaN,NR,--,2006,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,NaN,NaN
840,840,Matt Albers,RP,38.0,Signed,Orioles,Red Sox,1.0,NR,"$875,000",2010,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,BAL,BOS
1439,1439,Matt Albers,RP,38.0,Signed,Indians,Astros,1.0,NR,"$2,450,000",2013,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,CLE,HOU
1799,1799,Matt Albers,RP,38.0,Signed,White Sox,White Sox,1.0,NR,"$2,250,000",2015,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,CHW,CHW
1963,1963,Matt Albers,RP,38.0,Signed,White Sox,Nationals,NaN,NR,Minor Lg,2016,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,CHW,WSH
2168,2168,Matt Albers,RP,38.0,Signed,Nationals,Brewers,2.0,46,"$5,000,000",2017,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,WSH,MIL
2649,2649,Matt Albers,RP,38.0,FA,Brewers,--,NaN,NR,--,2019,https://www.espn.com/mlb/player/_/id/28525/mat...,True,28525,MIL,NaN


## 讀取 簽約資料中球員歷年表現

In [4]:
ESPN_stats_path = r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_player_stats_all_20210417_14.12.52.csv"
ESPN_stats_df = read_data_dir_fPathExt(ESPN_stats_path)

ESPN_bio_path = r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_player_bio_all_20210417_15.23.45.csv"
ESPN_bio_df = read_data_dir_fPathExt(ESPN_bio_path)

  >> 讀取:: G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_player_stats_all_20210417_14.12.52.csv
  >> 讀取:: G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\ESPN_player_bio_all_20210417_15.23.45.csv


In [5]:
# 格式轉換
ESPN_stats_df["playerID"] = ESPN_stats_df["playerID"].astype("str")
ESPN_bio_df["playerID"] = ESPN_bio_df["playerID"].astype("str")
ESPN_FA_df["playerID"] = ESPN_FA_df["playerID"].astype("str")

ESPN_stats_df["season"] = ESPN_stats_df["season"].astype("str")
ESPN_FA_df["YEAR"] = ESPN_FA_df["YEAR"].astype("str")

### ESPN_FA_df 球隊 正規化

In [6]:
print(f"ORG TEAM nunique: {ESPN_FA_df['ORG TEAM'].nunique()}\n\
NEW TEAM nunique: {ESPN_FA_df['NEW TEAM'].nunique()}\n\
Team nunique: {ESPN_stats_df['Team'].nunique()}")

ORG TEAM nunique: 34
NEW TEAM nunique: 32
Team nunique: 37


In [7]:
# ESPM MLB 球隊代號
url = "https://www.espn.com/mlb/teams"
response = rs.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
_as = soup.find_all('a')

team_dict = {}
for a in _as:
    try:
        if "/mlb/team/_/name/" in a['href']:
            teamFullName = a['href'].split("/")[-1].replace("-"," ").title()
            teamID = a['href'].split("/")[-2].upper()
            team_dict[teamFullName] = teamID
            
    except:
            pass

print(len(team_dict))
# team_dict

30


In [8]:
def team_ID_finder(team_name):
    for s_name in team_name.split(" "):
        for full_name, teamID in team_dict.items():
            if s_name in full_name:
                return teamID
    return np.nan

In [9]:
ESPN_FA_df.loc[:, "ORG_TEAM_ID"] = ESPN_FA_df.loc[:, "ORG TEAM"].apply(team_ID_finder)
ESPN_FA_df.loc[:, "NEW_TEAM_ID"] = ESPN_FA_df.loc[:, "NEW TEAM"].apply(team_ID_finder)

In [10]:
# 檢查那些縮寫有問題
team_dict_inverse = {v: k for k, v in team_dict.items()}

for t in ESPN_stats_df['Team'].unique():
    try:
        team_dict_inverse[t]
    except:
        print(t)

CWS
MON
FLA
WAS
CAL
ANA
CHI


In [86]:
# team_dict_inverse

In [11]:
# 球隊名稱清理

def team_ID_cleaner(teamID):
    if teamID == "FLA":
        return "MIA"
    
    if (teamID == "CWS"):
        return "CHW"
    
    if (teamID == "ANA") | (teamID == "CAL"):
        return "LAA"
    
    if (teamID == "MON") | (teamID == "WAS"):
        return "WSH"
    
    if teamID == "CHI":
        return np.nan
    
    return teamID


#### 篩選掉 Team有錯誤的球員 1筆 (id: 23283, Mike Jacobs)

In [12]:
ESPN_stats_df['Team_clean'] = ESPN_stats_df['Team'].apply(team_ID_cleaner)
ESPN_stats_df = ESPN_stats_df[pd.notna(ESPN_stats_df["Team_clean"])].reset_index(drop=True)
print(len(ESPN_stats_df))

15999


In [13]:
print(f"ORG TEAM(ID) nunique: {ESPN_FA_df['ORG_TEAM_ID'].nunique()}\n\
NEW TEAM(ID) nunique: {ESPN_FA_df['NEW_TEAM_ID'].nunique()}\n\
Team(clean) nunique: {ESPN_stats_df['Team_clean'].nunique()}")

ORG TEAM(ID) nunique: 30
NEW TEAM(ID) nunique: 30
Team(clean) nunique: 30


In [14]:
# 以 Team_clean 取代 Team
ESPN_stats_df['Team'] = ESPN_stats_df['Team_clean'].copy(deep=True)

In [15]:
# 檢查FA資料中球隊缺值情況
print(len(ESPN_FA_df))
print(f"ORG_TEAM_ID null: {ESPN_FA_df.loc[:, 'ORG_TEAM_ID'].isna().sum()}\nNEW_TEAM_ID null: {ESPN_FA_df.loc[:, 'NEW_TEAM_ID'].isna().sum()}")

mask_org = pd.isna(ESPN_FA_df.loc[:, "ORG_TEAM_ID"])
mask_new = pd.isna(ESPN_FA_df.loc[:, "NEW_TEAM_ID"])
print(ESPN_FA_df.loc[mask_org, "ORG TEAM"].unique(), ESPN_FA_df.loc[mask_new, "NEW TEAM"].unique())

2910
ORG_TEAM_ID null: 46
NEW_TEAM_ID null: 599
['--' 'Japan' 'Cuba'] ['--' 'Japan']


### 最後出賽年 (TODO 待討論)

In [16]:
player_max_season_df = ESPN_stats_df[["playerID","season"]].groupby(["playerID"], as_index=False).max()
player_max_season_df["season"].value_counts()

2021    178
2019    124
2018    103
2017     99
2020     99
2011     88
2013     88
2010     88
2009     82
2014     81
2015     77
2016     76
2012     74
2008     72
2007     71
2006     31
2005      4
Name: season, dtype: int64

In [ ]:
#TODO

### 將bio 欄位(POS、DOB、Birthplace、Type)合併入stats 

In [17]:
# 挑選需要的欄位
cols_pic = [
#     '_idx',
    'season',
    'Team',
    'GP',
#     'GS',
#     'W',
#     'L',
#     'W%',
    'WAR',
    'ERA',
#     'WHIP',
#     'IP',
#     'K',
#     'BB',
#     'K/BB',
#     'H',
#     'R',
#     'ER',
#     'SV',
#     'HLD',
#     'BLSV',
    'playerID',
    'Yrs',
#     'AB',
#     '2B',
#     '3B',
#     'HR',
#     'RBI',
#     'HBP',
#     'SO',
#     'SB',
#     'CS',
#     'AVG',
#     'OBP',
#     'SLG',
    'OPS'
]

ESPN_stats_bio_df = ESPN_stats_df[cols_pic].merge(ESPN_bio_df[["playerID", "POS", "DOB", "Birthplace", "TYPE"]], how="left", on=["playerID"])
print(len(ESPN_stats_df), len(ESPN_stats_bio_df))

15999 15999


In [18]:
ESPN_bio_df["TYPE"].value_counts()

pitch    728
bat      725
Name: TYPE, dtype: int64

#### 檢查TYPE是否正確(主職(投手沒OPS、打者沒ERA)) >> OK 

In [18]:
# mask_check = (ESPN_stats_bio_df["TYPE"].isin(["pitch"]))&(pd.notna(ESPN_stats_bio_df["OPS"]))
# ESPN_stats_bio_df[mask_check]

# mask_check = (ESPN_stats_bio_df["TYPE"].isin(["bat"]))&(pd.notna(ESPN_stats_bio_df["ERA"]))
# ESPN_stats_bio_df[mask_check]

### 守備位置正規化

In [19]:
def POS_cleaner(value):
    if "Starting Pitcher" in value:
        return "SP"
    
    elif ("Relief Pitcher" in value) | ("Pitcher" in value):
        return "RP"
    
    elif ("Fielder" in value) | ("Outfielder" in value):
        return "OF"

    elif "Shortstop" in value:
        return "SS"

    elif "First Baseman" in value:
        return "1B"

    elif "Second Baseman" in value:
        return "2B"

    elif "Third Baseman" in value:
        return "3B"

    elif "Catcher" in value:
        return "C"
    
    elif "Designated Hitter" in value:
        return "DH"
    
    else:
        return np.nan

In [20]:
ESPN_stats_bio_df["POS_c"] = ESPN_stats_bio_df["POS"].apply(POS_cleaner)

In [21]:
ESPN_stats_bio_df[pd.isna(ESPN_stats_bio_df["POS_c"])]["POS"].unique()

array(['DOB'], dtype=object)

In [22]:
ESPN_stats_bio_df[pd.isna(ESPN_stats_bio_df["POS_c"])]

,season,Team,GP,WAR,ERA,playerID,Yrs,OPS,POS,DOB,Birthplace,TYPE,POS_c
14857,2018,MIA,31,0.0,NaN,33619,1,0.0,DOB,12/30/1988,"Neenah, WI",bat,NaN


#### 篩選掉沒有正確POS資訊的選手 1名 (id: 33619)

In [23]:
ESPN_stats_bio_df = ESPN_stats_bio_df[pd.notna(ESPN_stats_bio_df["POS_c"])].reset_index(drop=True)
print(len(ESPN_stats_bio_df))

15998


In [24]:
print(ESPN_stats_bio_df["POS_c"].isna().sum())
ESPN_stats_bio_df["POS_c"].value_counts()

0


RP    4899
SP    2843
OF    2747
C     1387
1B    1050
2B    1016
3B     925
SS     670
DH     461
Name: POS_c, dtype: int64

### 清理DOB，計算年齡

In [25]:
ESPN_stats_bio_df["DOB"].unique()

array(['1/20/1983', '6/18/1966', '7/3/1966', ..., '5/30/1990 (30)',
       '1/12/1991 (30)', '8/30/1984 (36)'], dtype=object)

In [26]:
ESPN_stats_bio_df["DOB"] = ESPN_stats_bio_df["DOB"].apply(lambda s: s.split(" ")[0])
ESPN_stats_bio_df["DOB_fmt"] = pd.to_datetime(ESPN_stats_bio_df["DOB"], format='%m/%d/%Y', errors='coerce')
ESPN_stats_bio_df["Age"] = ESPN_stats_bio_df["season"].astype("int") - ESPN_stats_bio_df["DOB_fmt"].dt.year +1

## 表現資料串簽約資料 (球員ID、年分、原球隊)

In [27]:
# 串檔
FA_cols = ["playerID", "YEAR", "ORG_TEAM_ID", "NEW_TEAM_ID", "contract_Yrs", "DOLLARS", "STATUS"]
ESPN_stats_bio_FA_df = ESPN_stats_bio_df.merge(ESPN_FA_df[FA_cols], how="left", 
    left_on=["playerID", "season", "Team"], right_on=["playerID","YEAR","ORG_TEAM_ID"])

In [28]:
print(len(ESPN_stats_bio_df),len(ESPN_stats_bio_FA_df))

15998 15998


In [29]:
ESPN_stats_bio_FA_df["ORG_TEAM_ID"].notna().sum() / len(ESPN_stats_bio_FA_df)

0.1678959869983748

# 合併後變數處裡與分割

## 處理薪水， 將合約薪水攤到每一年去(TODO )

光有FA合約年的簽約薪資無法拚湊出每一年的薪資(新合約發生在成為FA前)，比對BR的資料出入太大(ex:Matt Albers ESPN ID: 28525)

In [30]:
mask_dont = (ESPN_stats_bio_FA_df["DOLLARS"].str.contains('Minor|-', regex=True)) | (pd.isna(ESPN_stats_bio_FA_df["DOLLARS"]))
ESPN_stats_bio_FA_df.loc[mask_dont, "DOLLARS"] = np.nan
ESPN_stats_bio_FA_df.loc[~mask_dont, "DOLLARS"] = ESPN_stats_bio_FA_df.loc[~mask_dont, "DOLLARS"].replace('[\$,]', '', regex=True).astype(float)
ESPN_stats_bio_FA_df["DOLLARS_M"] = ESPN_stats_bio_FA_df["DOLLARS"].apply(lambda n: n/(10**6) if isinstance(n,float) else np.nan)
ESPN_stats_bio_FA_df["DOLLARS_M"].describe()

count    1387.000000
mean       15.100686
std        32.473158
min         0.380000
25%         2.000000
50%         4.500000
75%        13.000000
max       330.000000
Name: DOLLARS_M, dtype: float64

In [31]:
# ESPN_stats_bio_FA_df[ESPN_stats_bio_FA_df["playerID"].isin(["28525"])]

In [ ]:
#TODO

## 分割資料: 1.每人每年在各球隊(已有:ESPN_stats_bio_FA_df)、2.每人每年整體(若單年有跨球隊表現會從 ESPN球員 Split分頁中找到整體值)

In [42]:
# ESPN_stats_bio_FA_df[ESPN_stats_bio_FA_df["playerID"].isin(["28525"])]

### 找出有單年度跨球隊表現的球員季度 (2006之後)

In [32]:
palyer_season_team_count_df = ESPN_stats_bio_FA_df.groupby(["playerID","season", "TYPE"], as_index=False)["Team"].count()
palyer_season_team_count_GOAL_df = palyer_season_team_count_df[(palyer_season_team_count_df["Team"]>1)&(palyer_season_team_count_df["season"].astype(int)>=2006)]

### 回ESPN 球員 Split 分頁 找到當年度整體表現 **(已執行過不再執行)**

#### ERA、OPS 的公式算法? 

In [33]:
# import time

# playerID_lst = []
# year_lst = []
# type_lst = []
# goal_col_name_lst = []
# goal_col_value_lst = []
# for _id, year, _type in tqdm(palyer_season_team_count_GOAL_df[["playerID", "season", "TYPE"]].values):
    
#     if _type == "pitch":
#         type_for_url = "pitching"
#         goal_col_name = "ERA"
    
#     if _type == "bat":
#         type_for_url = "batting"
#         goal_col_name = "OPS"

#     url = f"https://www.espn.com/mlb/player/splits/_/id/{_id}/year/{year}/category/{type_for_url}"
#     try:
#         s = pd.read_html(url)
#         df_temp = s[1].copy(deep=True)
#         df_temp.columns = s[1].loc[0,:].values
#         df_temp.drop(index=[0], inplace=True)
#         goal_col_value = float(df_temp.loc[1,goal_col_name])

#         playerID_lst.append(_id)
#         year_lst.append(year)
#         type_lst.append(_type)
#         goal_col_name_lst.append(goal_col_name)
#         goal_col_value_lst.append(goal_col_value)
        
#     except:
#         print(url)
    
#     time.sleep(1)

ERROR! Session/line number was not unique in database. History logging moved to new session 364


In [254]:
# GOAL_df = pd.DataFrame(
#     {'playerID': playerID_lst,
#      'season': year_lst,
#      'TYPE': type_lst,
#      'goal_col_name': goal_col_name_lst,
#      'goal_col_value': goal_col_value_lst
#     })

In [228]:
# # 輸出備份
# formtoCsv(GOAL_df, "GOAL_df")

 ==輸出備份csv==::: GOAL_df_20210417_21.08.00.csv


In [34]:
# 讀取備份
GOAL_df = read_data_dir_fPathExt(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\GOAL_df_20210417_21.08.00.csv")
GOAL_df.rename(columns={"type":"TYPE"},inplace=True)
GOAL_df["playerID"] = GOAL_df["playerID"].astype(str)
GOAL_df["season"] = GOAL_df["season"].astype(str)

  >> 讀取:: G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\GOAL_df_20210417_21.08.00.csv


### mutilTeam_count>1 串GOAL_df，後在與 mutilTeam_count=1 合併

In [35]:
palyer_season_team_count_GOAL_df.rename(columns={"Team":"mutilTeam_count"})
ESPN_stats_bio_FA_df = ESPN_stats_bio_FA_df.merge(palyer_season_team_count_GOAL_df.rename(columns={"Team":"mutilTeam_count"})
                                                 , how="left",on=["playerID", "season", "TYPE"])

ESPN_stats_bio_FA_df.loc[:,"mutilTeam_count"] = ESPN_stats_bio_FA_df.loc[:,"mutilTeam_count"].fillna(1)
print(len(ESPN_stats_bio_FA_df))

15998


In [36]:
temp_split_mutil_df = ESPN_stats_bio_FA_df[ESPN_stats_bio_FA_df["mutilTeam_count"]>1].reset_index(drop=True)
temp_split_one_df = ESPN_stats_bio_FA_df[ESPN_stats_bio_FA_df["mutilTeam_count"]==1].reset_index(drop=True)

In [37]:
# 加總 GP、WAR
cols = [
 'season',
#  'Team',
 'GP',
 'WAR',
#  'ERA',
 'playerID',
#  'Yrs',
#  'OPS',
#  'POS',
#  'DOB',
#  'Birthplace',
 'TYPE',
#  'POS_c',
#  'DOB_fmt',
#  'Age',
#  'YEAR',
#  'ORG_TEAM_ID',
#  'NEW_TEAM_ID',
#  'contract_Yrs',
#  'DOLLARS',
#  'STATUS',
#  'DOLLARS_M',
#  'mutilTeam_count'
]

temp_split_mutil_gp_df = temp_split_mutil_df[cols].groupby(['playerID','season','TYPE']).agg(
    {
            "GP":{
                "GP": "sum",
            },

            "WAR":{
                "WAR":"sum",
            }
    }).reset_index()

temp_split_mutil_gp_df.columns = ['playerID','season','TYPE', 'GP' , 'WAR']

# temp_split_mutil_gp_df

C:\Users\Daguan\miniconda3\lib\site-packages\pandas\core\groupby\generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


#### temp_split_mutil_gp_df 串 GOAL_df

In [38]:
# bat
temp_split_mutil_gp_bat_df = temp_split_mutil_gp_df[temp_split_mutil_gp_df["TYPE"].isin(["bat"])].merge(GOAL_df[["playerID","season","TYPE","goal_col_value"]]
                                                                          , how="left", on=["playerID","season","TYPE"])
temp_split_mutil_gp_bat_df.rename(columns={"goal_col_value":"OPS"},inplace=True)

# pitch
temp_split_mutil_gp_pitch_df = temp_split_mutil_gp_df[temp_split_mutil_gp_df["TYPE"].isin(["pitch"])].merge(GOAL_df[["playerID","season","TYPE","goal_col_value"]]
                                                                          , how="left", on=["playerID","season","TYPE"])
temp_split_mutil_gp_pitch_df.rename(columns={"goal_col_value":"ERA"},inplace=True)

In [39]:
temp_split_mutil_gp_m_df = pd.concat([temp_split_mutil_gp_bat_df, temp_split_mutil_gp_pitch_df], axis=0,sort=False).reset_index(drop=True)
print(len(temp_split_mutil_gp_m_df))

1088


In [40]:
#串回個人欄位
temp_split_mutil_gp_m_df = temp_split_mutil_gp_m_df.merge(ESPN_stats_bio_FA_df[["playerID","season","Yrs","Birthplace","POS_c","Age"]].drop_duplicates(
                                                            subset=["playerID","season"],keep="first")
                                                          , how="left",on=["playerID","season"])
print(len(temp_split_mutil_gp_m_df))

1088


In [41]:
#串回合約資料
temp_split_mutil_gp_m_df = temp_split_mutil_gp_m_df.merge(ESPN_stats_bio_FA_df[["playerID","season",'YEAR','ORG_TEAM_ID', 'NEW_TEAM_ID', 'contract_Yrs'
                                                                               ,'DOLLARS', 'STATUS','DOLLARS_M']].sort_values(by=["YEAR"],ascending=[1]
                                                            ).drop_duplicates(
                                                            subset=["playerID","season"],keep="first")
                                                          , how="left",on=["playerID","season"])
print(len(temp_split_mutil_gp_m_df))

1088


#### 與 mutilTeam_count=1 合併

In [42]:
# temp_split_mutil_gp_m_df.drop_duplicates(subset=["playerID","season"],keep="first")

In [43]:
#部分2006年以前的會撐開，但等等就拿掉了
ESPN_stats_bio_FA_no_team_gp_df = pd.concat([temp_split_mutil_gp_m_df, temp_split_one_df[temp_split_mutil_gp_m_df.columns.to_list()+["mutilTeam_count"]]], axis=0,sort=False).sort_values(
    by=["playerID","season"],ascending=[1,1]).reset_index(drop=True)
print(len(ESPN_stats_bio_FA_no_team_gp_df))

14864


## 延伸變數製作

### 合約年 dummy 製作

In [44]:
ESPN_stats_bio_FA_df["is_FA_contract_year"] = ESPN_stats_bio_FA_df["YEAR"].apply(lambda y: pd.notna(y))
ESPN_stats_bio_FA_no_team_gp_df["is_FA_contract_year"] = ESPN_stats_bio_FA_no_team_gp_df["YEAR"].apply(lambda y: pd.notna(y))

### 年齡平方

In [45]:
ESPN_stats_bio_FA_df["Age_sq"] = ESPN_stats_bio_FA_df["Age"] * ESPN_stats_bio_FA_df["Age"]
ESPN_stats_bio_FA_no_team_gp_df["Age_sq"] = ESPN_stats_bio_FA_no_team_gp_df["Age"] * ESPN_stats_bio_FA_no_team_gp_df["Age"]

### ERA 數值清理

In [46]:
def str_to_float_cleaner(str_value):
    try:
        if "inf" in str_value:
            return np.nan
        
        return float(str_value)
        
    except:
        return np.nan

In [47]:
ESPN_stats_bio_FA_df["ERA"] = ESPN_stats_bio_FA_df["ERA"].apply(str_to_float_cleaner)
ESPN_stats_bio_FA_no_team_gp_df["ERA"] = ESPN_stats_bio_FA_no_team_gp_df["ERA"].apply(str_to_float_cleaner)

### 數值ID (playerID_num、season_num) 

In [48]:
ESPN_stats_bio_FA_df["playerID_num"] = ESPN_stats_bio_FA_df["playerID"].astype(int)
ESPN_stats_bio_FA_no_team_gp_df["playerID_num"] = ESPN_stats_bio_FA_no_team_gp_df["playerID"].astype(int)

ESPN_stats_bio_FA_df["season_num"] = ESPN_stats_bio_FA_df["season"].astype(int)
ESPN_stats_bio_FA_no_team_gp_df["season_num"] = ESPN_stats_bio_FA_no_team_gp_df["season"].astype(int)

# 篩選表現2006年以後 & 大聯盟年資6年以上 輸出

### 有team版

In [49]:
mask_pic = (ESPN_stats_bio_FA_df["season_num"]>=2006) & (ESPN_stats_bio_FA_df["Yrs"]>=6)
ESPN_stats_bio_FA_pic_df = ESPN_stats_bio_FA_df[mask_pic].reset_index(drop=True)
print(len(ESPN_stats_bio_FA_df),len(ESPN_stats_bio_FA_pic_df))

15998 7039


#### 區分 bat、pitch 輸出

In [51]:

col_output_bat = [
 'season',
 'season_num',
 'Team',
 'GP',
 'WAR',
#  'ERA',
 'playerID',
 'playerID_num',
 'Yrs',
 'OPS',
#  'POS',
#  'DOB',
#  'Birthplace',
 'TYPE',
 'POS_c',
#  'DOB_fmt',
 'Age',
 'Age_sq',
 'YEAR',
 'ORG_TEAM_ID',
 'NEW_TEAM_ID',
 'contract_Yrs',
#  'DOLLARS',
 'STATUS',
 'DOLLARS_M',
#  'mutilTeam_count',
 'is_FA_contract_year'
]

formtoXlsx(ESPN_stats_bio_FA_pic_df.loc[ESPN_stats_bio_FA_pic_df["TYPE"].isin(["bat"]),col_output_bat].reset_index(drop=True), "ESPN_haveTeam_bat")
ESPN_stats_bio_FA_pic_df.loc[ESPN_stats_bio_FA_pic_df["TYPE"].isin(["bat"]),col_output_bat].reset_index(drop=True).to_stata('ESPN_haveTeam_bat.dta')


col_output_pitch = [
 'season',
 'season_num',
 'Team',
 'GP',
 'WAR',
 'ERA',
 'playerID',
 'playerID_num',
 'Yrs',
#  'OPS',
#  'POS',
#  'DOB',
#  'Birthplace',
 'TYPE',
 'POS_c',
#  'DOB_fmt',
 'Age',
 'Age_sq',
 'YEAR',
 'ORG_TEAM_ID',
 'NEW_TEAM_ID',
 'contract_Yrs',
#  'DOLLARS',
 'STATUS',
 'DOLLARS_M',
#  'mutilTeam_count',
 'is_FA_contract_year'
]

formtoXlsx(ESPN_stats_bio_FA_pic_df.loc[ESPN_stats_bio_FA_pic_df["TYPE"].isin(["pitch"]),col_output_pitch].reset_index(drop=True), "ESPN_haveTeam_pitch")
ESPN_stats_bio_FA_pic_df.loc[ESPN_stats_bio_FA_pic_df["TYPE"].isin(["pitch"]),col_output_pitch].reset_index(drop=True).to_stata('ESPN_haveTeam_pitch.dta')

 ==輸出備份xlsx== ::: ESPN_haveTeam_bat_20210418_12.40.27.xlsx
 ==輸出備份xlsx== ::: ESPN_haveTeam_pitch_20210418_12.40.38.xlsx


###  no team gp版

In [52]:
mask_pic = (ESPN_stats_bio_FA_no_team_gp_df["season_num"]>=2006) & (ESPN_stats_bio_FA_no_team_gp_df["Yrs"]>=6)
ESPN_stats_bio_FA_no_team_gp_pic_df = ESPN_stats_bio_FA_no_team_gp_df[mask_pic].reset_index(drop=True)
print(len(ESPN_stats_bio_FA_no_team_gp_df),len(ESPN_stats_bio_FA_no_team_gp_pic_df))

14864 6129


#### 區分 bat、pitch 輸出

In [53]:

col_output_bat = [
 'season',
 'season_num',
 'GP',
 'WAR',
#  'ERA',
 'playerID',
 'playerID_num',
 'Yrs',
 'OPS',
#  'POS',
#  'DOB',
#  'Birthplace',
 'TYPE',
 'POS_c',
#  'DOB_fmt',
 'Age',
 'Age_sq',
 'YEAR',
 'ORG_TEAM_ID',
 'NEW_TEAM_ID',
 'contract_Yrs',
#  'DOLLARS',
 'STATUS',
 'DOLLARS_M',
#  'mutilTeam_count',
 'is_FA_contract_year'
]

formtoXlsx(ESPN_stats_bio_FA_no_team_gp_pic_df.loc[ESPN_stats_bio_FA_no_team_gp_pic_df["TYPE"].isin(["bat"]),col_output_bat].reset_index(drop=True), "ESPN_noTeam_bat")
ESPN_stats_bio_FA_no_team_gp_pic_df.loc[ESPN_stats_bio_FA_no_team_gp_pic_df["TYPE"].isin(["bat"]),col_output_bat].reset_index(drop=True).to_stata('ESPN_noTeam_bat.dta')

col_output_pitch = [
 'season',
 'season_num',
 'GP',
 'WAR',
 'ERA',
 'playerID',
 'playerID_num',
 'Yrs',
#  'OPS',
#  'POS',
#  'DOB',
#  'Birthplace',
 'TYPE',
 'POS_c',
#  'DOB_fmt',
 'Age',
 'Age_sq',
 'YEAR',
 'ORG_TEAM_ID',
 'NEW_TEAM_ID',
 'contract_Yrs',
#  'DOLLARS',
 'STATUS',
 'DOLLARS_M',
#  'mutilTeam_count',
 'is_FA_contract_year'
]

formtoXlsx(ESPN_stats_bio_FA_no_team_gp_pic_df.loc[ESPN_stats_bio_FA_no_team_gp_pic_df["TYPE"].isin(["pitch"]),col_output_pitch].reset_index(drop=True), "ESPN_noTeam_pitch")
ESPN_stats_bio_FA_no_team_gp_pic_df.loc[ESPN_stats_bio_FA_no_team_gp_pic_df["TYPE"].isin(["pitch"]),col_output_pitch].reset_index(drop=True).to_stata('ESPN_noTeam_pitch.dta')

 ==輸出備份xlsx== ::: ESPN_noTeam_bat_20210418_12.40.46.xlsx
 ==輸出備份xlsx== ::: ESPN_noTeam_pitch_20210418_12.40.54.xlsx


In [60]:
# have_team_bat = read_data_dir_fPathExt(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\有team_bat_20210417_22.43.03.csv")
# have_team_pitch = read_data_dir_fPathExt(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\有team_pitch_20210417_22.43.03.csv")

# no_team_bat = read_data_dir_fPathExt(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\no_team_gp_bat_20210417_22.43.08.csv")
# no_team_pitch = read_data_dir_fPathExt(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\2.資料\6.ESPN_free_agent\no_team_gp_pitch_20210417_22.43.08.csv")